In [6]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix

# 1. Создаём случайную матрицу рейтингов (пользователи × товары)
np.random.seed(42)
users = 6
items = 8
ratings = np.random.randint(0, 6, size=(users, items))  # 0-5
print("Исходная матрица ")
print(ratings)

# 2. Исключаем неактивных пользователей (все нули)
active_mask = np.any(ratings > 0, axis=1)
ratings = ratings[active_mask, :]

# Исключаем низкорейтинговые товары (средний рейтинг < 2)
avg_item_ratings = np.true_divide(ratings.sum(axis=0), (ratings != 0).sum(axis=0), 
                                  where=(ratings != 0))
avg_item_ratings = np.nan_to_num(avg_item_ratings)  # заменяем NaN на 0
high_rating_mask = avg_item_ratings >= 2
ratings = ratings[high_rating_mask]

print("\nФильтрация\n")
print(ratings)

Исходная матрица 
[[3 4 2 4 4 1 2 2]
 [2 4 3 2 5 4 1 3]
 [5 5 1 3 4 0 3 1]
 [5 4 3 0 0 2 2 1]
 [3 3 5 5 5 2 3 3]
 [0 2 4 2 4 0 1 3]]

Фильтрация

[3 4 2 4 4 1 2 2 2 4 3 2 5 4 1 3 5 5 1 3 4 3 1 5 4 3 2 2 1 3 3 5 5 5 2 3 3
 2 4 2 4 1 3]


Координатный формат


In [7]:
coo = coo_matrix(ratings)
print("row indices:", coo.row)
print("col indices:", coo.col)
print("data:", coo.data)

row indices: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
col indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]
data: [3 4 2 4 4 1 2 2 2 4 3 2 5 4 1 3 5 5 1 3 4 3 1 5 4 3 2 2 1 3 3 5 5 5 2 3 3
 2 4 2 4 1 3]


Сжатый строковый формат

In [8]:
csr = csr_matrix(ratings)
print("data:", csr.data)
print("indices:", csr.indices)
print("indptr:", csr.indptr)

data: [3 4 2 4 4 1 2 2 2 4 3 2 5 4 1 3 5 5 1 3 4 3 1 5 4 3 2 2 1 3 3 5 5 5 2 3 3
 2 4 2 4 1 3]
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]
indptr: [ 0 43]


ELLPACK

In [12]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix

# 1. Случайная матрица рейтингов
np.random.seed(42)
users = 6
items = 8
ratings = np.random.randint(0, 6, size=(users, items))
print("=== Исходная матрица рейтингов ===")
print(ratings)

# 2. Удаляем неактивных пользователей (все оценки = 0)
active_mask = np.any(ratings > 0, axis=1)
ratings = ratings[active_mask, :]

# Удаляем низкорейтинговые товары (средний рейтинг < 2)
avg_item_ratings = np.true_divide(ratings.sum(axis=0), (ratings != 0).sum(axis=0), where=(ratings != 0))
avg_item_ratings = np.nan_to_num(avg_item_ratings)
high_rating_mask = avg_item_ratings >= 2
ratings = ratings[high_rating_mask]

# Гарантируем двумерность (на случай, если осталась одна строка)
if ratings.ndim == 1:
    ratings = ratings.reshape(1, -1)

print("\n=== После фильтрации (активные пользователи и товары со средним ≥ 2) ===")
print(ratings)

# 3. Координатный формат (COO)
coo = coo_matrix(ratings)
print("\n=== Координатный формат (COO) ===")
print("row indices:", coo.row)
print("col indices:", coo.col)
print("data:", coo.data)

# 4. Сжатый строковый формат (CSR)
csr = csr_matrix(ratings)
print("\n=== Сжатый строковый формат (CSR) ===")
print("data:", csr.data)
print("indices:", csr.indices)
print("indptr:", csr.indptr)

# 5. Формат ELLPACK
max_nnz = np.max(np.count_nonzero(ratings, axis=1))
rows, cols = ratings.shape
ell_values = np.zeros((rows, max_nnz), dtype=int)
ell_col_idx = np.zeros((rows, max_nnz), dtype=int)

for i in range(rows):
    nz_cols = np.nonzero(ratings[i])[0]
    nz_vals = ratings[i, nz_cols]
    ell_values[i, :len(nz_vals)] = nz_vals
    ell_col_idx[i, :len(nz_cols)] = nz_cols

print("\n=== Формат ELLPACK ===")
print("values:\n", ell_values)
print("column indices:\n", ell_col_idx)
print(f"Максимум ненулевых в строке: {max_nnz}")


=== Исходная матрица рейтингов ===
[[3 4 2 4 4 1 2 2]
 [2 4 3 2 5 4 1 3]
 [5 5 1 3 4 0 3 1]
 [5 4 3 0 0 2 2 1]
 [3 3 5 5 5 2 3 3]
 [0 2 4 2 4 0 1 3]]

=== После фильтрации (активные пользователи и товары со средним ≥ 2) ===
[[3 4 2 4 4 1 2 2 2 4 3 2 5 4 1 3 5 5 1 3 4 3 1 5 4 3 2 2 1 3 3 5 5 5 2 3
  3 2 4 2 4 1 3]]

=== Координатный формат (COO) ===
row indices: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
col indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]
data: [3 4 2 4 4 1 2 2 2 4 3 2 5 4 1 3 5 5 1 3 4 3 1 5 4 3 2 2 1 3 3 5 5 5 2 3 3
 2 4 2 4 1 3]

=== Сжатый строковый формат (CSR) ===
data: [3 4 2 4 4 1 2 2 2 4 3 2 5 4 1 3 5 5 1 3 4 3 1 5 4 3 2 2 1 3 3 5 5 5 2 3 3
 2 4 2 4 1 3]
indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]
indptr: [ 0 43]

=== Формат ELLP